# Raw data preparation

This notebook explains the data pre-processing pipeline on the public EURO 2020 dataset provided by StatsBomb. The following steps are performed:

- Create a local SQlite database for storing the data
- Convert StatsBomb events to SPADL actions
- Store SPADL actions and metadata in SQLite database


In [ ]:
from pathlib import Path
"""
How does go from statsbomb to spadl?
"""
def _flatten_id(d):
    newd = {}
    extra = {}
    for k, v in d.items():
        if isinstance(v, dict):
            if "id" in v and "name" in v:
                newd[k + "_id"] = v["id"]
                newd[k + "_name"] = v["name"]
            else:
                extra[k] = v
        else:
            newd[k] = v
        print(newd)
    newd["extra"] = extra
    return newd
from statsbombpy import sb
import pandas as pd
obj = list(sb.events(3795107, fmt="dict").values())
eventsdf = pd.DataFrame(_flatten_id(e) for e in obj)
eventsdf.columns

In [15]:
from socceraction.data.statsbomb import StatsBombLoader
data_loader = StatsBombLoader(getter="remote")#"getter": "remote"
events = data_loader.events(game_id = 3795107, load_360 = True)
import socceraction.spadl as spadl
spadl.statsbomb.convert_to_actions(events, 782)

/home/lz80/un-xPass/.venv/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/home/lz80/un-xPass/.venv/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,game_id,original_event_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,type_id,result_id,bodypart_id,action_id
0,3795107,cac7b0b7-e051-4266-874a-37d5185c8a4e,1,1.0,782,3289.0,52.058824,34.430380,46.941176,37.615190,0,1,5,0
1,3795107,5c1ef827-e053-44f8-a0f0-b312a3ca093d,1,1.0,782,5642.0,46.941176,37.615190,44.470588,39.853165,21,1,0,1
2,3795107,ba26ec0b-3274-481c-90d1-c58bd71bc81f,1,2.0,782,5642.0,44.470588,39.853165,29.205882,53.797468,0,1,5,2
3,3795107,1783a21c-bb70-4353-bed4-5883f66a1f68,1,4.0,782,3077.0,29.205882,53.797468,28.235294,56.207595,21,1,0,3
4,3795107,f8b6772a-4272-4aa1-8eda-184d2a8248a8,1,6.0,782,3077.0,28.235294,56.207595,35.382353,67.483544,0,1,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2193,3795107,91c6f928-680e-4f83-a003-4889a177efe8,2,3074.0,782,5632.0,101.029412,16.612658,101.294118,16.612658,21,1,0,2193
2194,3795107,6688f547-a021-41b1-a454-91ec7255c970,2,3074.0,782,5632.0,101.294118,16.612658,99.264706,27.372152,1,0,5,2194
2195,3795107,06d05275-9d28-41bf-8812-976814b2ec59,2,3074.0,914,6954.0,100.058824,26.597468,105.000000,0.000000,18,1,1,2195
2196,3795107,864e6afd-6772-4d61-875d-995d52ffc18a,2,3092.0,782,5630.0,105.000000,0.000000,99.529412,24.617722,5,1,5,2196


In [11]:
events["team_id"]

0       782
1       914
2       914
3       782
4       782
       ... 
3612    782
3613    914
3614    782
3615    914
3616    782
Name: team_id, Length: 3617, dtype: int64

In [ ]:
# Disable private API warnings
import warnings
from statsbombpy.api_client import NoAuthWarning
warnings.filterwarnings(action="ignore", category=NoAuthWarning, module='statsbombpy')

In [ ]:
from unxpass.databases import SQLiteDatabase
from unxpass.visualization import plot_action

## Configure leagues and seasons to download and convert

First, we define the StatsBomb IDs of the competitions and seasons that should be downloaded.

In [ ]:
datasets = [
    # Full EURO 2020 dataset
    #{ "getter": "remote", "competition_id":  55, "season_id": 43 }
    # BEL v ITA at EURO2020 (enable for a quick test run)
    #{ "getter": "remote", "competition_id":  53, "season_id": 106, "game_id": 3835322 }
    # You can also import a local dataset
    { "getter": "local", "root": "/home/lz80/un-xPass/notebooks/test_data", "competition_id":  53, "season_id": 106, "game_id": 3847567}
]

## Configure database

Next, we define were the processed data should be stored.

In [ ]:
DB_PATH = Path("../stores/weuros.sql")
db = SQLiteDatabase(DB_PATH)

## Import data

Now we can download, convert and save each dataset.

In [ ]:
for dataset in datasets:
    db.import_data(**dataset)

## Access data

The SQLite database now provides an interface for conveniently accessing the data.

In [ ]:
# List of games included in the database
df_games = db.games()
df_games.head()

In [ ]:
# Dataframe with all SPADL actions + 360 snapshots for a particular game
df_actions = db.actions(game_id=3835322)
actions = df_actions.head()
actions[actions["original_event_id"] == '3eb6a801-b69f-4a2f-8a81-aa25a8466c80']
actions

In [ ]:
sample = (3795107, 2)
plot_action(df_actions.loc[sample])

In [ ]:
db.close()